In [1]:
import django
import os
import json
import os.path

In [2]:
os.environ['DJANGO_SETTINGS_MODULE'] = 'django_settings'
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
django.setup()

In [3]:
from boto3.session import Session
import boto3
import botocore
from loader import load_credential
import random
import string
from PIL import Image
from PIL import ExifTags
from io import BytesIO
from django.core.files.uploadedfile import InMemoryUploadedFile
import natsort
import csv

In [4]:
######### upload Clothes Image to S3 #########

from kraken.models import *

In [5]:
excel_data_path = os.getenv('HOME') +'/Desktop/picture_data/size.csv'

In [6]:
seodaemun = []
header = []
rownum = 0
a=[]
with open(excel_data_path, 'r', encoding='utf-8') as f:
    csv_data = csv.reader(f)
    
    for row in csv_data:
        if rownum == 0:
            header = row
        else:
            a.append(row)
        rownum +=1

In [7]:
all_data = {}
for data in a:
    d_id = data[0]
    old_file_name = d_id + ' (1).jpg'
    string = ''
    new = {}
    
    for index, value in enumerate(data):
        if index == 0:
            pass
        elif index == 1:
            new['total'] = float(value)
        elif index == 2:
            new['chest_cross_section'] = float(value)
        elif index == 3:
            new['bottom_cross_section'] = float(value)
        elif index == 4:
            new['sleeve'] = float(value)
        elif index == 5:
            new['shoulder_cross_section'] = float(value)
        elif index == 6:
            new['sleeve_cross_section'] = float(value)
        elif index == 7:
            new['neck_cross_section'] = float(value)
        elif index == 8: # color
            new['color'] = value
        elif index == 11: # spread
            if value:
                new['spread'] = True
        else:
            if value:
                string += value
    new['old_name'] = string
    all_data[old_file_name]=new

In [ ]:
all_data

In [1]:
# UPLOAD CODE

In [ ]:
active_version = DataVersion.objects.get(is_active=True)
for key in all_data:
    old_file_name = key
    dataset = Dataset.objects.get(old_file_name=old_file_name)
    size_data, _ = Size.objects.get_or_create(version=active_version, dataset=dataset)
    remark_data = {}
    for feature in all_data[key]:
        if not feature in ['color', 'spread', 'old_name']:
            n_feature = SizeFeatures.objects.get(name=feature)
            value = all_data[key][feature]
            SizeTag.objects.create(size_data=size_data, features=n_feature, value=value)
        elif feature in ['color', 'spread']:
            remark_data[feature]=all_data[key][feature]
        elif feature == 'old_name':
            dataset.old_name = all_data[key][feature]
            dataset.save()
    if remark_data:
        RemarkTag.objects.create(dataset=dataset, **remark_data)